# segment

* 将测试集进行分割

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import pickle
import datetime

%matplotlib inline

from contextlib import contextmanager

@contextmanager
def timer(name):
    start_time = time.time()
    yield
    print(f'[{name} done in {time.time() - start_time:.2f} s]')

/home/kawayi-4/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
with timer('read pred'):
    toBePredict = pd.read_csv('data/toBePredicted_forUser.csv',index_col=False)
    toBePredict['realTime'] = pd.to_datetime('2017-'+toBePredict['O_DATA']+' '+toBePredict['predHour'],format='%Y-%m-%d %H:%M:%S')

[read pred done in 0.14 s]


In [3]:
assigment = pd.DataFrame()
with timer('read assigment csv'):
    for day in ['25','26','27','28','29','30','31']:
        temp = pd.read_csv("data/train201710"+day+".csv", sep=",")
        temp['O_TIME'] = pd.to_datetime('2017-10-'+day+' '+temp['O_TIME'],format='%Y-%m-%d %H:%M:%S')
        temp = temp.groupby(['O_LINENO', 'O_TERMINALNO']).apply(lambda x:x.sort_values(by='O_TIME', ascending=True))
        assigment = pd.concat([assigment,temp])
assigment['O_DATA'] = assigment['O_TIME'].dt.day

[read assigment csv done in 363.98 s]


In [4]:
from math import sin, cos, sqrt, atan2, radians,acos,pi

def distance(coord1, coord2):
    R = 6373.0
    lat1 = radians(coord1[0])
    lon1 = radians(coord1[1])
    lat2 = radians(coord2[0])
    lon2 = radians(coord2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c
    return d

def get_last_station(bus1):
    # bus1[(bus1['O_SPEED'] == 0) & (bus1['O_LONGITUDE'] != 0.0) & (bus1['O_LATITUDE'] != 0.0)]
    len = bus1.shape[0]
    d = 0
    for i in range(len-1,0,-1):
        start_bus = bus1.iloc[i-1]
        station = start_bus['O_NEXTSTATIONNO']
        loc = [start_bus['O_LATITUDE'], start_bus['O_LONGITUDE']]

        next_bus = bus1.iloc[i]
        next_station = next_bus['O_NEXTSTATIONNO']
        next_loc = [next_bus['O_LATITUDE'], next_bus['O_LONGITUDE']]

        if (next_station == station):            
            d = d + distance(loc, next_loc)
        if (next_station > station):
            return next_station-1,bus1.iloc[i]['O_UP'],bus1.iloc[i],d
        elif (next_station < station):
            return next_station-1,bus1.iloc[i]['O_UP'],bus1.iloc[i-1],d

In [93]:
df = pd.DataFrame()
count = 0

for row in example.itertuples():
    import datetime
    line = row[2] # 线路
    carNo = row[3] # 车编号
    currtime = row[8] # 此时时间
    twominute = datetime.timedelta(minutes=-60) + currtime
    
    bus1 = assigment[(assigment['O_TERMINALNO']==carNo) & (assigment['O_LINENO']==line)]
    bus1 = bus1[(bus1['O_TIME']<currtime) & (bus1['O_TIME']>twominute)]
    
    if get_last_station(bus1) is None:
        #如果发生下面都不动了
        try:
            res = bus1.iloc[-1]
            lastStation_NO = res[11]-1
            lastStation_Time = res[2]
            LONGITUDE = res[3]
            LATITUDE = res[4]
            UP = res[9]
            d = -1
            with open('problems.txt','a+') as f:
                f.write(str(tuple(row))+'\n')
                print(str(tuple(row))+'\n')
        except:
            res = -1
            lastStation_NO = -1
            lastStation_Time = -1
            LONGITUDE = -1
            LATITUDE = -1
            UP = row[7]
            d = -1
            with open('problems.txt','a+') as f:
                f.write(str(tuple(row))+'\n')
                print(str(tuple(row))+'\n')
    else:
        last_station_NO, up, last_station_info, dis= get_last_station(bus1)
        # 如果
        lastStation_NO = last_station_NO
        lastStation_Time = last_station_info[2]
        LONGITUDE = last_station_info[3]
        LATITUDE = last_station_info[4]
        UP = up
        d = dis
    
    
    """
    O_DATA O_LINENO O_TERMINALNO predHour pred_start_stop_ID pred_end_stop_ID 
    
    realTime lastStation lastStation_Time LONGITUDE LATITUDE UP d
    
    """
    DATA = row[1]
    LINE = row[2]
    TERMINALNO = row[3]
    predHour = row[4]
    pred_start_stop_ID = row[5]
    pred_end_stop_ID = row[6]
    realTime = row[8]
    
#     lastStation_NO = last_station_NO
#     lastStation_Time = last_station_info[2]
#     LONGITUDE = last_station_info[3]
#     LATITUDE = last_station_info[4]
#     UP = last_station_info[9]
    
    
    temp = pd.DataFrame({
        'DATA':[DATA],
        'LINE':[LINE],
        'TERMINALNO':[TERMINALNO],
        'predHour':[predHour],
        'pred_start_stop_ID':[pred_start_stop_ID],
        'pred_end_stop_ID':[pred_end_stop_ID],
        'realTime':[realTime],
        'lastStation_NO':[lastStation_NO],
        'lastStation_Time':[lastStation_Time],
        'LONGITUDE':[LONGITUDE],
        'LATITUDE':[LATITUDE],
        'UP':[UP],
        'distance':[d]
    })
    df = pd.concat([df,temp])
    print('finishded {}\n'.format(row[0]))

(38, '10-26', 47, 963840, '7:00:00', 2, 20, 0, Timestamp('2017-10-26 07:00:00'))

finishded 38

finishded 19968



In [ ]:
df.to_csv('toBePredict_0605.csv',index=False)

In [146]:
toBePredicted_0605 = pd.read_csv('toBePredict_0605.csv')

In [147]:
toBePredicted_0605.head(2)

,DATA,LATITUDE,LINE,LONGITUDE,TERMINALNO,UP,distance,lastStation_NO,lastStation_Time,predHour,pred_end_stop_ID,pred_start_stop_ID,realTime
0,10-28,39.1098,678,117.249,904388,0,0.417984,11,2017-10-28 13:59:01,14:00:00,33,12,2017-10-28 14:00:00
1,10-26,39.1130,13,117.213,902540,1,0.203901,6,2017-10-26 16:58:57,17:00:00,19,7,2017-10-26 17:00:00


In [131]:
index = toBePredicted_0605[toBePredicted_0605['UP']==-1].index.values

In [150]:
for i in index:
    v = toBePredict.iloc[i, 6]
    toBePredicted_0605.set_value(i,'UP',v)

/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [153]:
toBePredicted_0605.to_csv('toBePredicted_0607.csv',index=False)

In [164]:
toBePredicted_0607 = pd.read_csv('toBePredicted_0607.csv')

In [166]:
df = pd.DataFrame()
for row in toBePredicted_0607.itertuples():
    DATA = row[1]
    LATITUDE = row[2]
    LINE = row[3]
    LONGITUDE = row[4]
    TERMINALNO = row[5]
    UP = row[6]
    distance = row[7]
    lastStation_NO = row[8]
    lastStation_Time = row[9]
    predHour = row[10]
    pred_end_stop_ID = row[11]
    pred_start_stop_ID = row[12]
    realTime = row[13]
    tmp = pd.DataFrame()
    
    if lastStation_NO>pred_start_stop_ID:
        lastStation_NO = -1
    
    if lastStation_NO !=-1:
        start = min(pred_start_stop_ID-1,lastStation_NO)
    else:
        start = pred_start_stop_ID-1
        lastStation_Time = realTime = row[13]

        
    for p in range(start,pred_end_stop_ID):
        if p==start:
            d = distance
        else:
            d = 0
        res = pd.DataFrame({'DATA':[DATA],
                            'LATITUDE':[LATITUDE],
                            'LINE':[LINE],
                            'LONGITUDE':[LONGITUDE],
                            'TERMINALNO':[TERMINALNO],
                            'UP':[UP],
                            'lastStation_NO':[lastStation_NO],
                            'lastStation_Time':[lastStation_Time],
                            'predHour':[predHour],
                            'pred_end_stop_ID':[p+1],
                            'pred_start_stop_ID':[p],
                            'realTime':[realTime],
                            'distance':[d]})
        
        
        
        tmp = pd.concat([tmp,res])
    df = pd.concat([df,tmp])
    if row[0] % 5000==0:
        print('finished %d \n'% row[0])

finished 0 

finished 5000 

finished 10000 

finished 15000 

finished 20000 

finished 25000 

finished 30000 



In [ ]:
df.to_csv('toBePredicted_0607_segment.csv')